In [1]:
from mm_datasets import * 
import os 

config_folder = "/home/allanz/lmm/lmm_synthetic/mm_train/dataset_configs"
datasets = []
for file in os.listdir(config_folder):
    file_path = os.path.join(config_folder, file)
    print(file_path)
    datasets.append(LazySupervisedDataset(file_path))

/home/allanz/miniconda3/envs/vlm/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2025-01-24 18:21:32.461 | INFO     | mm_datasets:__init__:179 - Dataset size: 10
2025-01-24 18:21:32.471 | INFO     | mm_datasets:__init__:179 - Dataset size: 10
2025-01-24 18:21:32.481 | INFO     | mm_datasets:__init__:179 - Dataset size: 10
2025-01-24 18:21:32.486 | INFO     | mm_datasets:__init__:179 - Dataset size: 10
2025-01-24 18:21:32.496 | INFO     | mm_datasets:__init__:179 - Dataset size: 10
2025-01-24 18:21:32.506 | INFO     | mm_datasets:__init__:179 - Dataset size: 10
2025-01-24 18:21:32.511 | INFO     | mm_datasets:__init__:179 - Dataset size: 10


/home/allanz/lmm/lmm_synthetic/mm_train/dataset_configs/original.json
/home/allanz/lmm/lmm_synthetic/mm_train/dataset_configs/grid.json
/home/allanz/lmm/lmm_synthetic/mm_train/dataset_configs/sub_sampling.json
/home/allanz/lmm/lmm_synthetic/mm_train/dataset_configs/distinct.json
/home/allanz/lmm/lmm_synthetic/mm_train/dataset_configs/original_align.json
/home/allanz/lmm/lmm_synthetic/mm_train/dataset_configs/sub_sampling_align.json
/home/allanz/lmm/lmm_synthetic/mm_train/dataset_configs/distinct_align.json


In [2]:
# Original 
datasets[0][0]

{'image': <PIL.Image.Image image mode=RGB size=256x256>,
 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].",
 'conversations': [['What object is in row 0, column 0? ', 'dog'],
  ['What object is in row 0, column 1? ', 'airplane'],
  ['What object is in row 2, column 0? ', 'frog'],
  ['What object is in row 0, column 2? ', 'airplane'],
  ['What object is in row 1, column 2? ', 'frog'],
  ['What object is in row 1, column 1? ', 'dog'],
  ['What object is in row 2, column 2? ', 'ship'],
  ['What object is in row 1, column 0? ', 'ship'],
  ['What object is in row 2, column 1? ', 'frog']]}

In [3]:
# Grid
datasets[1][0]

{'image': <PIL.Image.Image image mode=RGB size=256x256>,
 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].",
 'conversations': [['',
   '| dog | airplane | airplane |\n| ship | dog | frog |\n| frog | frog | ship |']]}

In [4]:
# Subsample
datasets[2][0]

{'image': <PIL.Image.Image image mode=RGB size=256x256>,
 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].",
 'conversations': [['What object is in row 0, column 2? ', 'airplane'],
  ['What object is in row 0, column 0? ', 'dog'],
  ['What object is in row 1, column 0? ', 'ship']]}

In [5]:
# Distinct 
for i in range(4):
    print(datasets[3][i])

{'image': <PIL.Image.Image image mode=RGB size=256x256 at 0x7F4B28505870>, 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].", 'conversations': [['What object is in row 2, column 0? ', 'frog'], ['What object is in row 0, column 2? ', 'airplane'], ['What object is in row 1, column 0? ', 'ship']]}
{'image': <PIL.Image.Image image mode=RGB size=256x256 at 0x7F4B28505FC0>, 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].", 'conversations': [['What object is in row 2, column 2? ', 'ship'], ['What object is in row 0, column 0? ', 'dog'], ['What object is in row 2, column 0? ', 'frog']]}
{'image': <PIL.Image.Image image mode=RGB size=256x256 at 0x7F4B285045B0>, 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].", 'conversations': [['What object is in row 1, column 2? ', 'frog'], ['What object is in row 1, c

In [7]:
# Original Align 
datasets[4][0]

{'image': <PIL.Image.Image image mode=RGB size=256x256>,
 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].",
 'conversations': [['',
   'What object is in row 0, column 0? dog\nWhat object is in row 0, column 1? airplane\nWhat object is in row 2, column 0? frog\nWhat object is in row 0, column 2? airplane\nWhat object is in row 1, column 2? frog\nWhat object is in row 1, column 1? dog\nWhat object is in row 2, column 2? ship\nWhat object is in row 1, column 0? ship\nWhat object is in row 2, column 1? frog\n']]}

In [8]:
# Sub Sample Align
datasets[5][0]

{'image': <PIL.Image.Image image mode=RGB size=256x256>,
 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].",
 'conversations': [['',
   'What object is in row 1, column 2? frog\nWhat object is in row 0, column 0? dog\nWhat object is in row 2, column 1? frog\n']]}

In [9]:
# Distinct Align
for i in range(4):
    print(datasets[6][i])

{'image': <PIL.Image.Image image mode=RGB size=256x256 at 0x7F4B28504640>, 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].", 'conversations': ['What object is in row 1, column 0? shipWhat object is in row 1, column 2? frogWhat object is in row 1, column 1? dog\n']}
{'image': <PIL.Image.Image image mode=RGB size=256x256 at 0x7F4B285044F0>, 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].", 'conversations': ['What object is in row 2, column 0? frogWhat object is in row 1, column 2? frogWhat object is in row 0, column 2? airplane\n']}
{'image': <PIL.Image.Image image mode=RGB size=256x256 at 0x7F4B285055D0>, 'prompt': "The grid above is size 3 by 3. Each cell contains an object from ['dog', 'ship', 'airplane', 'frog'].", 'conversations': ['What object is in row 0, column 2? airplaneWhat object is in row 1, column 1? dogWhat object is in row 2, column 0? frog\n']